In [13]:
# pip install Bio

import statements:

In [14]:
import Features_Modules2 as fm

In [15]:
import pandas as pd
import boto3
import os
from scipy.stats import spearmanr
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler
# import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from scipy.stats import kendalltau, pearsonr, spearmanr
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
import joblib
import tempfile

%matplotlib inline

Models

In [16]:
df = fm.read_clean_data('clean_train.csv')
model_data, features, scaler = fm.pre_process_data(df, [(1,3)])
# model_data.columns

columns:
                     0         1         2         3         4
a            0.615201 -0.377777 -0.098502 -0.470868 -0.222624
aa           0.000000  0.000000  0.000000  0.000000  0.000000
aaa          0.000000  0.000000  0.000000  0.000000  0.000000
aac          0.000000  0.000000  0.000000  0.000000  0.000000
aad          0.000000  0.000000  0.000000  0.000000  0.000000
...               ...       ...       ...       ...       ...
sheet_pct    0.888101 -0.572175 -0.572076  0.282371 -1.566996
aromaticity -0.401409  1.243221 -1.404423 -0.853947  0.016042
instability  0.733810 -1.030558  1.298767 -0.434823 -0.663036
charge      -2.643637 -1.245348 -0.125341  0.127627  0.019773
gravy       -0.252528  0.145358 -1.041713  0.636547  1.105112

[8432 rows x 5 columns]


In [17]:
# Base model for random tree
regr = RandomForestRegressor(n_estimators=50, n_jobs=5, random_state=42)

Y = model_data.label
X = model_data.drop('label',axis=1)

In [18]:
regr.fit(X,Y)
r2 = regr.score(X,Y)

yhat = regr.predict(X)
spear = fm.get_spearman(Y, yhat)
print("correlation with actual rank:", spear[0],"\nR^2 value for model:", r2)

correlation with actual rank: 0.9627005287576968 
R^2 value for model: 0.936156281163213


Read in test data (from test/train split)

In [19]:
df_test = fm.read_clean_data('clean_test.csv')
df_test.head()

,protein_sequence,pH,label
0,MGNSSSSSSRINKKSESMEIRSKPNRSSVSGEFNKSKSMMDMSSKS...,7.0,45.6
1,MKFLQIIPVLLSLTSTTLAQSFCSSASHSGQSVKETGNKVGTIGGV...,7.5,55.0
2,MNKRDAINHIRQFGGRLDRFRKSVGGSGDGGRNAPKRAKSSPKPVN...,7.0,56.1
3,MTSLVPGAGLLPIPTSSPLTAVSSLGVSLSSLGAIPAAALDPNITT...,7.0,50.3
4,MSSNKYHHGDLKASLIEAANTILLRDGADALSLRAIAAEVGVSHMA...,7.0,45.1


In [20]:
test1 = fm.prediction_prep(df_test, features, scaler)

In [21]:
prediction = regr.predict(test1)
print("correlation with actual rank:", fm.get_spearman(prediction, df_test.label)[0],
      "\nR^2 value for model:", regr.score(test1,df_test.label),
     "\nMSE for model:", mean_squared_error(prediction,df_test.label))

correlation with actual rank: 0.5104645293688199 
R^2 value for model: 0.5242660483404402 
MSE for model: 68.64561164257137


PERSIST THE MODEL

In [22]:
bucket = 'tech-x-final-project'
MODEL_FOLDER = 'models'

In [23]:
my_region = boto3.session.Session().region_name
print(my_region)
s3 = boto3.client('s3')

us-east-1


In [24]:
key1 = "models/model.pkl"
key2 = "models/scaler.pkl"
key3 = "models/features.pkl"
with tempfile.TemporaryFile() as fp:
    joblib.dump(regr, fp)
    fp.seek(0)
    s3.put_object(Key=key1, Body=fp.read(), Bucket=bucket)
    
with tempfile.TemporaryFile() as fp:
    joblib.dump(scaler, fp)
    fp.seek(0)
    s3.put_object(Key=key2, Body=fp.read(), Bucket=bucket)
    
with tempfile.TemporaryFile() as fp:
    joblib.dump(features, fp)
    fp.seek(0)
    s3.put_object(Key=key3, Body=fp.read(), Bucket=bucket)